In [118]:
%reset -f

# Gaussian process for geographic data (GP lab part 2)

**Machine Learning, University of Zaragoza, Ruben Martinez-Cantin**

In this lab, you will implement a Gaussian process for air quality data, obtained from the City Hall in their [open data site](https://www.zaragoza.es/sede/portal/datos-abiertos/servicio/catalogo/131).

  *adapted from a lab by Luis Montesano*

In [46]:
#@title install libraries
# !pip install GPy

In [119]:
#@title import libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from geopy import Nominatim, Photon
from PIL import Image
from pyproj import Transformer
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import GPy

# descomenta %matplotlib qt si prefieres plots interactivos
%matplotlib inline
# %matplotlib qt

## 0. Setup y Auxiliary Funtions

### 0.1 Coordinate transformation

The geographical coordinates we will use are given in degrees (*latitude* and *longitude*). Therefore, we need a way to transform them (or *project* them) to metric units (e.g. meters), as these are more suitable in the calculation of distances needed in **Gaussian processes**.

For these transformations, we will use the **PyProj** library. [ [code](https://github.com/pyproj4/pyproj) | [docs](https://pyproj4.github.io/pyproj/stable/) ].


&#9432; **EXTRA INFO**

These transformations are performed using the following **coordinate reference systems (CRS)**:
* EPSG:4326 (WGS84) as being the appropriate CRS for latitude and longitude coordinates (represented in degrees) [<a href="https://epsg.io/?q=4326">link</a>].
* EPSG:32630 (UTM, zona 30) as this CRS is used to express the coordinates in meters, projecting them to a local 2D plane. Zone 30 is the one corresponding to Zaragoza [<a href="https://www.ign.es/web/coordenadas-de-stations-ergnss">link</a>].


In [120]:
transformer_longlat_xy = Transformer.from_crs(4326, 32630, always_xy=True)


def from_longlat_to_xy(long, lat):
    """Transform longitude and latitude coordinates to x, y coordinates in
    CRS 32630 (UTM, zone 30)

    Args:
        long: (n,) array or float with longitude (degrees)
        lat: (n,) array or float with latitude (degrees)

    Returns:
        x: (n,) array or float with x coordinates in CRS 32630
        y: (n,) array or float with y coordinates in CRS 32630
    """
    return transformer_longlat_xy.transform(long, lat)


def from_xy_to_longlat(x, y):
    """Transform x, y coordinates in CRS 32630 (UTM, zone 30) to longitude and
    latitude coordinates

    Args:
        x: (n,) array or float with x coordinates in CRS 32630
        y: (n,) array or float with y coordinates in CRS 32630

    Returns:
        long: (n,) array or float with longitude (degrees)
        lat: (n,) array or float with latitude (degrees)
    """
    return transformer_longlat_xy.transform(x, y, direction="inverse")

### 0.2 Generation and data loading


First, we need to obtain the **geographic coordinates** (latitude and longitude) of the air quality measurement **stations** in Zaragoza.

Knowing the station location (street or address) for each station, we use `Photon` *geodecoder* to obtain an automated (albeit approximate) location of the stations using `OpenStreetMap` data. For that, we use the `Geopy` library [[*GH*](https://github.com/geopy/geopy) | [*Docs*](https://geopy.readthedocs.io/en/stable/)].

The address of each station can be found in the city hall [website](https://www.zaragoza.es/sede/portal/medioambiente/calidad-aire/). We store the data in a `Pandas` dataframe to simplify access and to show the results in a table form.

In [121]:
stations = {
    "Actur": "Calle Cineasta Carlos Saura, 50018",
    "El Picarral": "Av. San Juan de la Peña / C.S. Picarral, 50015",
    "Roger de Flor": "Calle Roger de Flor, 50017",
    "Jaime Ferran": "Calle Jaime Ferrán, 50014",
    "Renovales": "Paseo de Mariano Renovales, 50006",
    "Las Fuentes": "Calle María de Aragón, 50013",
    "Centro": "Calle José Luis Albareda, 50004",
    "Avda. Soria": "Avenida Ciudad de Soria, 50010, Zaragoza, España",
}

latitudes, longitudes = [], []
geolocator = Photon(user_agent="myGeocoder",timeout=100)
for name, address in stations.items():
    location = geolocator.geocode(address)
    latitudes.append(location.latitude)  # type: ignore
    longitudes.append(location.longitude)  # type: ignore

data = {
    "station": list(stations.keys()),
    "latitude": latitudes,
    "longitude": longitudes,
}
df_stations = pd.DataFrame(data)

df_stations

,station,latitude,longitude
0,Actur,41.675101,-0.888726
1,El Picarral,41.667799,-0.871634
2,Roger de Flor,41.650049,-0.917384
3,Jaime Ferran,41.665323,-0.862919
4,Renovales,41.633303,-0.891697
5,Las Fuentes,41.641563,-0.862971
6,Centro,41.649634,-0.885469
7,Avda. Soria,41.659058,-0.907933


Now, let's add the **metric coordinates**  $x, y$ (meters) corresponding to the *CRS UTM (zone 30)* of each station:

In [122]:
df_stations[["x", "y"]] = df_stations.apply(
    lambda row: from_longlat_to_xy(row.longitude, row.latitude),
    axis=1,
    result_type="expand",
)
df_stations

,station,latitude,longitude,x,y
0,Actur,41.675101,-0.888726,675741.966418,4.615857e+06
1,El Picarral,41.667799,-0.871634,677184.810141,4.615082e+06
2,Roger de Flor,41.650049,-0.917384,673423.593261,4.613018e+06
3,Jaime Ferran,41.665323,-0.862919,677917.177571,4.614825e+06
4,Renovales,41.633303,-0.891697,675608.180010,4.611211e+06
5,Las Fuentes,41.641563,-0.862971,677978.316702,4.612187e+06
6,Centro,41.649634,-0.885469,676082.517258,4.613037e+06
7,Avda. Soria,41.659058,-0.907933,674186.377457,4.614037e+06


Last, we load the file `calidad_aire_2023.csv` corresponding to the air quality measured as the *concentration* [$\mu g/m^3$] of several chemical contaminants, measured in each station through 2023.

If you are working locally in your system or you already have downloaded the file, you can skip this step.

In [51]:
#@title download data
#!wget https://raw.githubusercontent.com/rmcantin/gp_lab/main/datasets/calidad_aire_2023.csv

In [123]:
#@title load downloaded data
path_calidad = "calidad_aire_2023.csv"
df_air_q = pd.read_csv(path_calidad, sep=",")
df_air_q

,id,title,contaminante,nivel,value,fecha,year
0,220230224nox,Roger de Flor,nox,NaN,20.1,24-02-2023,2023
1,720230224no2,Jaime Ferran,no2,Buena,27.5,24-02-2023,2023
2,720230224sh2,Jaime Ferran,sh2,Buena,1.7,24-02-2023,2023
3,820230224so2,Renovales,so2,Buena,9.6,24-02-2023,2023
4,820230224co,Renovales,co,Buena,0.2,24-02-2023,2023
...,...,...,...,...,...,...,...
9995,820230903nox,Renovales,nox,NaN,6.2,03-09-2023,2023
9996,1020230903co,Centro,co,Buena,0.2,03-09-2023,2023
9997,1220230903no2,Actur,no2,Buena,10.3,03-09-2023,2023
9998,1120230905co,Avda. Soria,co,Buena,0.2,05-09-2023,2023


### 0.3 Interactive visualization

In the next code, we develop the class `MapPlotter`, based in [**Plotly**](https://plotly.com/graphing-libraries/) (using the [map visualization API](https://plotly.com/python/mapbox-layers/)) which defines auxiliar methods to visualize the lab session results.

In [124]:
class MapPlotter:
    """Interactive map visualization based on Plotly"""

    CENTER = {"lat": 41.65573, "lon": -0.88616}

    def __init__(self, zoom=12, center=None, showlegend=False, **kwargs):
        self.fig = go.Figure()
        self.fig.update_layout(
            mapbox_style="open-street-map",  # or "carto-positron",
            mapbox_zoom=zoom,
            mapbox_center=center or self.CENTER,
            margin={"r": 0, "t": 10, "l": 0, "b": 10},
            showlegend=showlegend,
            # autosize=True,
            **kwargs,
        )

    def add_longlat_points(
        self,
        lon,
        lat,
        s=15,
        color=None,
        text=None,
        edgecolor=None,
        name=None,
        adjust_center=True,
        **kwargs,
    ):
        if edgecolor is not None:
            self.fig.add_trace(
                go.Scattermapbox(
                    lat=lat,
                    lon=lon,
                    mode="markers",
                    marker_size=1.1 * s,
                    marker_color=edgecolor,
                    showlegend=False,
                    **kwargs,
                )
            )

        self.fig.add_trace(
            go.Scattermapbox(
                lat=lat,
                lon=lon,
                mode="markers",
                marker_size=s,
                marker_color=color,
                text=text,
                name=name,
                **kwargs,
            )
        )

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

    def add_density_heatmap(self, lon, lat, z, radius=10, adjust_center=True, **kwargs):
        self.fig.add_trace(
            go.Densitymapbox(
                lat=lat.ravel(),
                lon=lon.ravel(),
                z=z.ravel(),
                radius=radius,
                colorbar_title="μg/m³",
                **kwargs,
            )
        )

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

    def add_raster_heatmap(
        self,
        xgrid,
        ygrid,
        heatmap,
        cmap="Spectral_r",
        adjust_center=True,
        add_cbar=True,
        **kwargs,
    ):
        # rasterize the heatmap
        heatmap_ = (heatmap - heatmap.min()) / np.ptp(heatmap)
        heatmap_ = plt.get_cmap(cmap)(heatmap_)[..., :3]
        raster_hm = Image.fromarray((heatmap_ * 255).clip(0, 255).astype(np.uint8))

        # heatmap bounding box
        bbox = np.array(
            [
                [xgrid[0, 0], ygrid[0, 0]],  # top-left
                [xgrid[0, -1], ygrid[0, -1]],  # top-right
                [xgrid[-1, -1], ygrid[-1, -1]],  # bottom-right
                [xgrid[-1, 0], ygrid[-1, 0]],  # bottom-left
            ]
        )
        lon, lat = from_xy_to_longlat(bbox[:, 0], bbox[:, 1])
        coords = np.stack((lon, lat), axis=1)

        layers = [*self.fig.layout.mapbox.layers]  # type: ignore
        layers.append(
            {
                "sourcetype": "image",
                "source": raster_hm,
                "coordinates": coords,
                "below": "traces",
                **kwargs,
            }
        )
        self.fig.layout.mapbox.layers = layers  # type: ignore

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

        if add_cbar:
            # add colorbar via invisible scatter trace
            self.fig.add_trace(
                go.Scattermapbox(
                    lat=[None],
                    lon=[None],
                    mode="markers",
                    marker=dict(
                        colorscale=cmap,
                        color=heatmap,
                        cmin=heatmap.min(),
                        cmax=heatmap.max(),
                        showscale=True,
                        colorbar=dict(
                            title="μg/m³",
                            # titleside="right",
                            ticks="outside",
                        ),
                    ),
                )
            )

    def adjust_center(self, lon, lat):
        self.fig.update_layout(mapbox_center={"lat": lat, "lon": lon})

    def show(self):
        self.fig.show()

## 1. Gaussian process for geographical data

In this part of the lab, you must implement the **Gaussian process model** to interpolate the air quaility in Zaragoza and visualize the results obtained.

For this, you need to complete the following functions:

&#9432; **WARNING**

Some stations do not have concentration data for all the contaminants, because they lack the corresponding sensor. We recommend you to use the following contaminants which are available in all the stations:

* `nox`
* `no2`
* `no`
* `o3`

In [125]:
#@title Utils to process the data
def create_regular_grid(X, npts=100):
    """Coordinate mesh grid of where to predict the air quaility using GPs

    Args:
        X: (nobs, 2) array of the xy coordinates of the observations
        npts: number of points in width and height in the grid.

    Returns:
        xgrid: (npts, npts) array with x coordinates of the grid
        ygrid: (npts, npts) array with y coordinates of the grid
    """
    # grid limits
    x_min, x_max = X[..., 0].min(), X[..., 0].max()
    y_min, y_max = X[..., 1].min(), X[..., 1].max()

    # we add 10% padding/margin
    padding_x = (x_max - x_min) * 0.1
    padding_y = (y_max - y_min) * 0.1

    x_min -= padding_x
    x_max += padding_x
    y_min -= padding_y
    y_max += padding_y

    # regular grid
    x, y = np.linspace(x_min, x_max, npts), np.linspace(y_min, y_max, npts)
    xgrid, ygrid = np.meshgrid(x, y)
    return xgrid, ygrid

def load_air_quality_data(df_air_q, contaminant="o3", log_data=False):
    """Load air quality data for GP estimation in Zaragoza

    Args:
        df_air_q: Air quality DataFrame
        contaminant: str, contaminant name to estimate
        log_data: if True, print the observations of that contaminant in each
            station

    Returns:
        X: (n_stations, 2) array with the location of the stations
        measured_concentration: (n_stations, ) array with the observations for
            contaminant at the stations (average of 2023).
    """
    # first, we check that the contamint is in the DataFrame
    assert (
        contaminant in df_air_q["contaminante"].unique()
    ), "Contaminant not found"

    # we filter all data regarding the contamint
    data_contaminante = df_air_q[df_air_q["contaminante"] == contaminant]

    # As observation, we use the mean value along 2023 of each contamint in
    # each station
    observations = data_contaminante.groupby("title")["value"].mean()

    # We create an auxiliary dataframe where we add the observation to each station
    df_est = df_stations.copy()
    df_est["concen."] = df_est["station"].map(observations)
    assert (
        df_est["concen."].notnull().all()
    ), "Missing observations for some stations"

    if log_data:
        print(df_est)

    # Input data for the Gaussian process
    X = df_est[["x", "y"]].values    # location of the stations
    measured_concentrations = df_est["concen."].values

    return X, measured_concentrations

In [211]:
#@title Function to predict concentration based on a Gaussian process
#@markdown Complete the TODO sections


def air_quality_prediction(X, measured_concentrations, gp_kernel, npts_grid=100):
    """GP-based air quality estimation in Zaragoza

    Args:
        X: (n_stations, 2) array with the location of the stations
        measured_concentration: (n_stations, ) array with the observations for
            contaminant at the stations (average of 2023).
        gp_kernel: Kernel for the GP to train
        npts_grid: int, number of points in width and height in the grid where
            we are going to predict the air quality

    Returns:
        (xgrid, ygrid): tupla with all x,y coordinates of the grid in
            shape (npts, npts)
        pred_concentration: (npts, npts) array with the predicted mean value of
            the contamint at each point of the grid.
        sigma_concentration: (npts, npts) array with the predicted standard deviation
            of the contamint at each point of the grid.
    """

    # Gaussian process training using the observations
    # TODO: Add the training code here using GPy and gp_kernel
    #
    # IMPORTANT:
    #    Sometimes Python vectors are represented with an empty dimension (n, ).
    #    Many libraries consider verctors as a special case of a 1D matrix.
    #    Thus, we need to make the vector (n, 1). For that, you can use either:
    #     - numpy.atleast_2d
    #     - numpy.newaxis
    #    Check the documentation for each method. Note that (1, n) is a different
    #    matrix, but you can then transpose the result.
    # Reshape the measurements to a column vector
    X = np.atleast_2d(X)
    measured_concentrations = np.atleast_2d(measured_concentrations).T  # Make it (n_stations, 1)

    # Create and train the GP model using GPy
    model = GPy.models.GPRegression(X, measured_concentrations, gp_kernel)
    model.optimize_restarts(num_restarts=5)
    display(model)


    # Create a regular mesh to predict the air quality in all the city.
    xgrid, ygrid = create_regular_grid(X, npts=npts_grid)

    # Gaussian process prediction at the grid points
    # TODO: Using the trained model, predict the concentration mean and standar
    # deviation at the grid locations
    #
    # IMPORTANT:
    #     You might need to reshape the values of the grid. GPy expects a single
    #     matrix with size (n_predictions, input_dim). To reshape the grids you
    #     can use the following methods:
    #     -numpy.ravel
    #     -numpy.vstack
    #     and you might also need to transpose the results.
    # Reshape the grid for predictions

    Xgrid = np.vstack([xgrid.ravel(), ygrid.ravel()]).T  # Reshape grid to (n_predictions, 2)


    # Reshape the GP predictions to adjust it to the grid: shape (npts, npts)
    # This will be required for plotting. Right now we are returning random values
    # that you can use for checking the plots.
    #
    # TODO: replace the random values with the GP prediction.
    #
    # IMPORTANT:
    #      For reshape, you can direcly use numpy.reshape. Also, check that you
    #      return the standard deviation and not the variance.

    # Comment these two lines and make your own predictions
    # pred_concentration = np.random.rand(npts_grid, npts_grid)
    # std_concentration = np.random.rand(npts_grid, npts_grid)

    # Predict the mean and standard deviation at the grid points
    pred_mean, pred_var = model.predict(Xgrid)

    # Reshape the predicted mean and standard deviation to match grid shape
    pred_concentration = pred_mean.reshape((npts_grid, npts_grid))
    std_concentration = np.sqrt(pred_var).reshape((npts_grid, npts_grid))  # Convert variance to std deviation



    return (xgrid, ygrid), pred_concentration, std_concentration

## Process data and Kernel definition

Using GPy, you need to define the kernel. Try different kernels and combinations of them.

IMPORTANT: GPy uses gradient descent to learn the hyperparameters. This might fail if the hyperparameters are quite far from the optimal values as they get stuck in local minima. You can solve this in two ways:
- Either give an good initial value for the hyperparameters (order of magnitude). For example: as a rule of thumb, the lenghtscale should be in the same scale as the GP inputs and the variance should be in the same scale as the GP outputs.
- Alternatively, if we assume that the original data follows a Gaussian distribution $X \sim N(\mu_X, \sigma_X)$, then you can normalize your data such that the mean value is 0 and the variance is 1, that is, $newX \sim N(0,1)$. Then:
$$ newX = \frac{X - \mu_X}{\sigma_X}$$
If you train your GP with the normalize data, then, you need to remember to apply the same factor to the prediction location

$$ new\_prediction = \frac{original\_prediction - \mu_X}{\sigma_X}$$

**Data normalization** is always a good practice if done properly and improves the numerical performance in **all kind of problems**.

In [231]:
from sklearn.preprocessing import StandardScaler

def air_quality_prediction_of_contaminant (kernel, contaminant, npts_grid=100):
    
    # Load the data for the selected contaminant
    X, y = load_air_quality_data(df_air_q, contaminant=contaminant)

    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_normalized = scaler_X.fit_transform(X)
    y_normalized = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

    # Make predictions with the GP model in the normalized space
    _, concentration_prediction_normalized, concentration_std_normalized = air_quality_prediction(X_normalized, y_normalized, kernel, npts_grid=100)

    # Reshape predictions to match grid shape
    concentration_prediction = scaler_y.inverse_transform(concentration_prediction_normalized)
    concentration_std = scaler_y.scale_ * concentration_std_normalized

    # Create a grid in the normalized space
    xgrid_normalized, ygrid_normalized = create_regular_grid(X_normalized, npts=100)

    # Inverse transform the grid for plotting
    grid_original = scaler_X.inverse_transform(np.column_stack([xgrid_normalized.ravel(), ygrid_normalized.ravel()]))
    xgrid_original = grid_original[:, 0].reshape(xgrid_normalized.shape)
    ygrid_original = grid_original[:, 1].reshape(ygrid_normalized.shape)

    return xgrid_original, ygrid_original, concentration_prediction, concentration_std

RBF_kernel = GPy.kern.RBF(input_dim=2,variance=1.0,lengthscale=1.0)
Cosine_kernel = GPy.kern.Cosine(input_dim=2,variance=1.0,lengthscale=1.0)
Matern32_kernel = GPy.kern.Matern32(input_dim=2,variance=1.0,lengthscale=1.0)
RBF_Matern32_kernel = RBF_kernel + Matern32_kernel



# Searaching for best kernel

In [229]:
import GPy
from sklearn.preprocessing import StandardScaler
import numpy as np

# Initialize scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Define kernel types to evaluate
kernel_types = [
    ("RBF", GPy.kern.RBF),
    ("Matern32", GPy.kern.Matern32),
    ("Matern52", GPy.kern.Matern52),
    ("RationalQuadratic", GPy.kern.RatQuad)
]

# Loop over contaminants
results = {}
contaminants = ["o3", "nox", "no2", "no"]  # Example contaminant names

for contaminant in contaminants:
    print(f"\nEvaluating contaminant: {contaminant}")
    # Load data for current contaminant
    X, y = load_air_quality_data(df_air_q, contaminant=contaminant)
    
    # Normalize data
    X_normalized = scaler_X.fit_transform(X)
    y_normalized = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()
    
    # Store best model results
    best_log_likelihood = -np.inf
    best_kernel_name = None
    best_model = None
    
    for kernel_name, kernel_func in kernel_types:
        print(f"\nTrying kernel: {kernel_name}")
        
        # Initialize kernel with initial hyperparameters
        kernel = kernel_func(input_dim=2, variance=1.0, lengthscale=1.0)
        
        # Create and optimize GP model with restarts
        model = GPy.models.GPRegression(X_normalized, y_normalized[:, None], kernel)
        model.optimize_restarts(num_restarts=5, messages=False)
        
        # Retrieve log-likelihood
        log_likelihood = model.log_likelihood()
        print(f"Log-likelihood for {kernel_name}: {log_likelihood}")
        
        # Check if this is the best kernel so far
        if log_likelihood > best_log_likelihood:
            best_log_likelihood = log_likelihood
            best_kernel_name = kernel_name
            best_model = model
    
    # Save results for the contaminant
    results[contaminant] = {
        "best_kernel": best_kernel_name,
        "log_likelihood": best_log_likelihood,
        "model": best_model,
    }

    print(f"\nBest kernel for {contaminant}: {best_kernel_name} with log-likelihood {best_log_likelihood}")

# Final results summary
for contaminant, result in results.items():
    print(f"\nContaminant: {contaminant}")
    print(f"Best Kernel: {result['best_kernel']}")
    print(f"Log Likelihood: {result['log_likelihood']}")
    print(f"Variance: {result['model'].kern.variance.values[0]}")
    print(f"Lengthscale: {result['model'].kern.lengthscale.values[0]}")

mean_best_variance = np.mean([result['model'].kern.variance.values[0] for result in results.values()])
mean_best_lengthscale = np.mean([result['model'].kern.lengthscale.values[0] for result in results.values()])

print(f"\nMean Variance: {mean_best_variance}")
print(f"\nMean Lengthscale: {mean_best_lengthscale}")


Evaluating contaminant: o3

Trying kernel: RBF
Optimization restart 1/5, f = 11.285822403436566
Optimization restart 2/5, f = 11.285822409915436
Optimization restart 3/5, f = 11.285822403660957
Optimization restart 4/5, f = 11.285822398523988
Optimization restart 5/5, f = 11.285822407280342
Log-likelihood for RBF: -11.285822398523988

Trying kernel: Matern32
Optimization restart 1/5, f = 11.299703403807099
Optimization restart 2/5, f = 11.299703341031671
Optimization restart 3/5, f = 11.29970336471547
Optimization restart 4/5, f = 11.299703244759606
Optimization restart 5/5, f = 11.29970327419128
Log-likelihood for Matern32: -11.299703244759606

Trying kernel: Matern52
Optimization restart 1/5, f = 11.294589180843733
Optimization restart 2/5, f = 11.294589189766148
Optimization restart 3/5, f = 11.294589156033958
Optimization restart 4/5, f = 11.294589167329072
Optimization restart 5/5, f = 11.294589156407103
Log-likelihood for Matern52: -11.294589156033958

Trying kernel: RationalQua

## Visualization

Let's plot the prediction and standard deviation in two separate maps.


In [228]:
def plot_prediction_variance(xgrid, ygrid, concentration_prediction, concentration_std): # Initialize the map plotter
    fig = MapPlotter()

    # Add station points to the map
    fig.add_longlat_points(
        df_stations["longitude"],
        df_stations["latitude"],
        text=df_stations["station"],
        color="white",
        edgecolor="black",
    )

    # Add the mean prediction as a heatmap on the grid
    fig.add_raster_heatmap(
        xgrid, ygrid, concentration_prediction,
        cmap="Spectral_r",  # Customize color map if desired
        opacity=0.6
    )

    # Show the map with the predicted concentration mean
    fig.show()


    # Initialize the map plotter for standard deviation visualization
    fig_std = MapPlotter()

    # Add station points to this map
    fig_std.add_longlat_points(
        df_stations["longitude"],
        df_stations["latitude"],
        text=df_stations["station"],
        color="white",
        edgecolor="black",
    )

    # Add the standard deviation heatmap to the grid
    fig_std.add_raster_heatmap(
        xgrid, ygrid, concentration_std,
        cmap="Spectral_r",
        opacity=0.6
    )

    # Udpdate the legend of the colorbar of the add_raster_heatmap method
    fig_std.fig.update_traces(marker=dict(colorbar=dict(title="Std")))
    #Update the cmin and CMAX of the colorbar
    fig_std.fig.update_traces(
            go.Scattermapbox(
                mode="markers",
                marker=dict(
                    cmin=concentration_std.min(),
                    cmax=concentration_std.max(),
                    showscale=True,
                    colorbar=dict(
                        title="Std",
                        # titleside="right",
                        ticks="outside",
                    ),
                ),
            )
        )

    # Show the map with predicted concentration uncertainty
    fig_std.show()



# Predict and plot with mean best model as seed

In [233]:
#predict with best model
kernel = GPy.kern.RBF(input_dim=2, variance=mean_best_variance, lengthscale=mean_best_lengthscale)

for contaminant in contaminants:
    print(f"\nPredicting contaminant: {contaminant}")
    xgrid, ygrid, concentration_prediction, concentration_std = air_quality_prediction_of_contaminant(kernel, contaminant)
    plot_prediction_variance(xgrid, ygrid, concentration_prediction, concentration_std)




Predicting contaminant: o3
Optimization restart 1/5, f = 11.285822419075545
Optimization restart 2/5, f = 11.285822404499402
Optimization restart 3/5, f = 11.285822401271387
Optimization restart 4/5, f = 11.285822429305263
Optimization restart 5/5, f = 11.285822400872618


GP_regression.,value,constraints,priors
rbf.variance,0.9879936936493985,+ve,
rbf.lengthscale,0.27451412388558416,+ve,
Gaussian_noise.variance,2.6319778724724437e-05,+ve,



Predicting contaminant: nox
Optimization restart 1/5, f = 10.068100025584815
Optimization restart 2/5, f = 10.068099811588347
Optimization restart 3/5, f = 10.068099501997231
Optimization restart 4/5, f = 10.068099512966466
Optimization restart 5/5, f = 10.06809951435888


GP_regression.,value,constraints,priors
rbf.variance,1.3004898527038944,+ve,
rbf.lengthscale,1.0899866555378845,+ve,
Gaussian_noise.variance,3.985577008869428e-09,+ve,



Predicting contaminant: no2
Optimization restart 1/5, f = 11.142473858588009
Optimization restart 2/5, f = 11.142473858589362
Optimization restart 3/5, f = 11.142473858741909
Optimization restart 4/5, f = 11.142473858643715
Optimization restart 5/5, f = 11.14247385883806


GP_regression.,value,constraints,priors
rbf.variance,0.519493259095118,+ve,
rbf.lengthscale,0.6412092879452942,+ve,
Gaussian_noise.variance,0.4591231200002347,+ve,



Predicting contaminant: no
Optimization restart 1/5, f = 10.191431846236227
Optimization restart 2/5, f = 10.19143235369286
Optimization restart 3/5, f = 10.19143184369182
Optimization restart 4/5, f = 10.19143186522058
Optimization restart 5/5, f = 10.191431850157835


GP_regression.,value,constraints,priors
rbf.variance,0.9574705587850197,+ve,
rbf.lengthscale,0.7838242879293764,+ve,
Gaussian_noise.variance,7.930519727877624e-09,+ve,


# Your tasks

- Use different kernels to predict the concentration for each of the contaminants. Justify your selection.

- Explain how did you solve the problem with the hyperparameter learning.

- If you can invest in a new station. Where would you place it?
zona roja, menos datos

- Kernel Selection for Contaminants:
Based on empirical tests across all contaminants (NO2, O3, NO, NOx), the Radial Basis Function (RBF) kernel consistently yielded the most reliable predictions. 

- 	Hyperparameter Optimization Strategy:
To address challenges in hyperparameter tuning, particularly ensuring convergence to optimal values, we employed an iterative optimization approach using optimize_restarts with five restarts. This process was conducted separately for each contaminant, with each RBF kernel initialized using averaged parameter values derived from prior optimization runs. 

- 	Optimal Location for New Station Placement:
To maximize the informational gain from an additional monitoring station, we recommend situating it in an area with consistently high uncertainty (variance) across contaminants. Visual inspection of the variance plots across contaminants suggests that the Casablanca or Parque del Agua regions exhibit higher predictive uncertainty, indicating these locations could most benefit from additional data.